In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [ ]:
df_reviews = spark.read.csv('googleplaystore_user_reviews.csv',inferSchema=True, header =True)
df_play = spark.read.csv('googleplaystore.csv',inferSchema=True, header =True)

In [ ]:
data = spark.read.csv('googleplaystore.csv',inferSchema=True, header=True)


In [ ]:
df_play.printSchema()